In [5]:
import numpy as np
import pandas as pd

In [6]:
df = pd.read_csv('Dataset_Jan.csv')

In [7]:
df.head()

,Area,Year,Item,Area Harvested,Yield,Humidity,Wind Speed,Temperature,Production
0,Bangladesh,1961,Areca nuts,82600,7627,NaN,NaN,NaN,62995
1,Bangladesh,1961,Bananas,33600,132738,NaN,NaN,NaN,446000
2,Bangladesh,1961,Barley,29947,5768,NaN,NaN,NaN,17272
3,Bangladesh,1961,"Bastfibres, others",30900,11117,NaN,NaN,NaN,34350
4,Bangladesh,1961,"Beans, dry",68798,7236,NaN,NaN,NaN,49784


In [8]:
# df['Item'].value_counts()
df['Item'].nunique()

54

## OneHotEncoding using sklearn

In [9]:
from sklearn.model_selection import train_test_split
# first 5 cols as train as input, last col as output
X_train,X_test,y_train,y_test = train_test_split(df.iloc[:,0:5],df.iloc[:,-1],test_size=0.2,random_state=2)

In [10]:
X_train.head()

,Area,Year,Item,Area Harvested,Yield
2560,Bangladesh,2008,Lentils,72581,9856
2720,Bangladesh,2011,Jute,708723,21494
1346,Bangladesh,1985,"Vegetables, fresh nes",77023,62181
148,Bangladesh,1963,Sorghum,809,12559
2473,Bangladesh,2006,Sugar crops nes,20234,191756


In [11]:
from sklearn.preprocessing import OneHotEncoder

In [12]:
# object
# multicollinearity removal
# removing the catagories from the columns with drop first
ohe = OneHotEncoder(drop='first',sparse=False,dtype=np.int32)

In [13]:
X_train_new = ohe.fit_transform(X_train[['Area','Item']])
# we get the no. of cols depending on the categories
X_train_new

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [23]:
X_train_new.shape

(2548, 53)

In [14]:
X_test_new = ohe.transform(X_test[['Area','Item']])

In [15]:
X_test_new

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

In [16]:
# now to append everything together
data = np.hstack((X_train[['Year','Area Harvested','Yield']].values,X_train_new))
data

array([[    2008,    72581,     9856, ...,        0,        0,        0],
       [    2011,   708723,    21494, ...,        0,        0,        0],
       [    1985,    77023,    62181, ...,        0,        1,        0],
       ...,
       [    1990,     4014,    45279, ...,        0,        0,        0],
       [    2008,    46482,     8547, ...,        0,        0,        0],
       [    2008, 11279150,    41441, ...,        0,        0,        0]],
      dtype=int64)

In [20]:
X

NameError: name 'X' is not defined

In [17]:
# Transforming the data
from sklearn.preprocessing import MinMaxScaler
scalar = MinMaxScaler()
X_train_mlr = scalar.fit_transform(X_train_new)
X_test_mlr = scalar.transform(X_test_new)

### K-fold cross validation between all the models

In [42]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

In [43]:
x = data[:,0:5]
y = data[:,-1]
X_train_new, X_test_new, y_train, y_test = train_test_split(x,y,test_size=0.2)

In [44]:
svm = SVC()
svm.fit(X_train_new,y_train)
print(cross_val_score(svm, X_train_new, y_train, cv=5, scoring = 'accuracy').mean())

0.9911704485233898


In [45]:
knn =  KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_new, y_train)
print(cross_val_score(knn, X_train_new, y_train, cv=5, scoring = 'accuracy').mean())

0.9936250421544539


In [50]:
mlr = LinearRegression()
mlr.fit(X_train_mlr, y_train)

print(cross_val_score(mlr, X_train_mlr, y_test, cv=5, scoring = 'accuracy').mean())

ValueError: Found input variables with inconsistent numbers of samples: [2038, 510]